In [1]:
using Plots
using DelimitedFiles
using Random
using LinearAlgebra

In [2]:
function X0(m, bL, bU)
    if length(bL) == length(bU)
        # Número de parametros
        n = length(bL)
        # Inicializando la población
        Xr = zeros(m, n)
        # Añadiendo la informaciónd de cada individuo
        for i in 1:m
            Xr[i,:] = rand(n) .* (bU - bL) + bL
        end
    else
        println("No coincide el tamaño de los límites inferior y superior de los parametros")
    end
    return Xr
end

X0 (generic function with 1 method)

In [3]:
function Vmut(X, F, bL, bU, var)
    m, n = size(X)
    M = zeros(m, n)
    rnd = zeros(m, n)

    if var in ["d","D"]
        rnd .= rand(m)
    elseif var in ["j","J"]
        rnd = rand(m, n)
    end

    for i in 1:m
        r = shuffle(1:m)[1:3]
        M[i,:] = @. X[r[1],:] + F * rnd[i,:] * (X[r[2],:] - X[r[3],:])
        
        indices = abs.(M[i,:] - (bL + bU) ./ 2) .> (bU - bL) ./ 2
        M[i, indices] = bL[indices] + (bU[indices] - bL[indices]) .* rand(length(M[i,indices]))
    end
    return M
end

Vmut (generic function with 1 method)

In [4]:
function Cross(X, M, Cr)
    # Verificación
    if size(X) == size(M) && abs(Cr - 0.5) <= 0.5
        # Tamaño de las matrices
        m, n = size(X)
        U = zeros(m,n)
        rnd = rand(m, n)
        rndi = rand(1:n, m, n) 
        indexes = (1:n)' .* ones(m)
        # Matriz logica en donde se cumple la condicion 1
        cond1 = rnd .<= Cr
        # Matriz logica en donde se cumple la condicion 2
        cond2 = indexes .== rndi
        # Aplicando el operador "or" para cambiar los elementos que cumplen al menos una condicion
        U[cond1 .| cond2] = M[cond1 .| cond2]
        U[.~(cond1 .| cond2)] = X[.~(cond1 .| cond2)]
        return U

    elseif size(X) != size(M)
        println("La matriz de mutación y la de población inicial deben de tener el mismo tamaño")

    elseif abs(Cr - 0.5) > 0.5
        println("Cr debe ser un numero entre 0 y 1")
        
    end
end

Cross (generic function with 1 method)

In [5]:
function Selection(Fobj, X, U, caso)
    if size(X) == size(U)
        m, n = size(X)
        S = X
        # Creando donde guardar las funciones evaluadas en las poblaciones
        Fi = zeros(m) # Evaluada en la poblacion inicial
        Fc = zeros(m) # Evaluada en la poblacion cruzada
        # Evaluando la poblacion inicial y la cruzada en la función
        for i in 1:m
            Fi[i] = Fobj(X[i,:])
            Fc[i] = Fobj(U[i,:])
        end
        # Creando las mascaras
        if caso == "min"
            indice = Fc .< Fi
        elseif caso == "max"
            indice = Fc .> F
        end
        # Aplicando los cambios de acuerdo a las mascaras
        S[indice,:] = U[indice,:]
        return S

    elseif size(X) != size(U)
        println("La matriz de población inicial y la cruzada no tienen el mismo tamaño")
        
    end
end

Selection (generic function with 1 method)

In [6]:
function Best(fob, X, caso)
    m, n = size(X)
    t = zeros(m)
    for i in 1:m
        t[i] = fob(X[i,:])
    end
    if caso == "min"
        g = min(t...)
        
    elseif caso == "max"
        g = max(t...)
    end
    indices = X[t .== g,:][1,:]
    return g, indices
end

Best (generic function with 1 method)

# Cosine Mixture function

In [7]:
function Fobj1(X)
    return dot(X,X) - 0.1*sum(cos.(5*pi.*X))
end

Fobj1 (generic function with 1 method)

In [8]:
#Poniendo los datos
#Gerneraciones e individuos
batch_size = 1000
#Dimensionalidad
D = 30
#Límites de los parametros
bL = -1 .*ones(D)
bU = ones(D)

value = -0.1*D
error = 0.00001
#Argumentos de mutacion y cruza
F = 1.5
Cr = 0.3

0.3

In [9]:
initial = X0(batch_size, bL, bU)
i = 1
while true
    mutation = Vmut(initial,F,bL,bU,"d")
    crossed = Cross(initial,mutation,Cr)
    chosen = Selection(Fobj1,initial,crossed,"min")
    minimum, batch = Best(Fobj1, initial, "min")
    IJulia.clear_output(true)
    println("Iter: $i, value: $minimum, error: $(abs(minimum - value))")
    i = i+1
    if abs(minimum - value) <= error
        return minimum, batch
        break
    end
end


Iter: 538, value: -2.999990559934581, error: 9.440065419141774e-6


(-2.999990559934581, [1.39654e-6, -0.000183439, 6.83913e-5, 9.8216e-5, -0.000188542, 5.01192e-5, 8.91654e-5, -1.62338e-5, -0.000119132, 1.63876e-6  …  0.000264053, 0.000444061, -7.99051e-5, 8.59781e-5, -0.000150445, -0.000163526, -2.67669e-5, -3.38451e-5, 0.000253995, 3.61592e-5])

Valor de $-2.999992427252088$ con error de $7.572747911854805 \times 10^{-6}$

# Griewank function

In [10]:
function Fobj2(X)
    product = 1
    for i in 1:length(X)
        product = product * cos(X[i]/sqrt(i))
    end    
    return 1 + dot(X,X)/4000 - product
end

Fobj2 (generic function with 1 method)

In [11]:
#Poniendo los datos
#Gerneraciones e individuos
batch_size = 1000
#Dimensionalidad
D = 30
#Límites de los parametros
bL = -600 .*ones(D)
bU = 600 .*ones(D)

value = 0
error = 0.00001
#Argumentos de mutacion y cruza
F = 1.5
Cr = 0.3

0.3

In [12]:
initial = X0(batch_size, bL, bU)
i = 1
while true
    mutation = Vmut(initial,F,bL,bU,"d")
    crossed = Cross(initial,mutation,Cr)
    chosen = Selection(Fobj2,initial,crossed,"min")
    minimum, batch = Best(Fobj2, initial, "min")
    IJulia.clear_output(true)
    println("Iter: $i, value: $minimum, error: $(abs(minimum - value))")
    i = i+1
    if abs(minimum - value) <= error
        return minimum, batch
        break
    end
end


Iter: 855, value: 8.738938098495908e-6, error: 8.738938098495908e-6


(8.738938098495908e-6, [0.00105354, 0.0002389, -0.00128572, 0.0008805, -0.00246191, -0.00124519, -0.000821625, 0.000158104, 0.0014608, -0.000546081  …  -0.00697119, 0.005857, 0.0059488, 0.0031875, 0.00139886, 0.00130187, 0.000695671, 0.00188923, 0.00808289, 0.000844055])

Valor de $9.083586132274668 \times 10^{-6}$ con error de $9.083586132274668 \times 10^{-6}$

# Inverted cosine wave function

In [13]:
function Fobj3(X)
    result = 0
    for i in 1:length(X)-1
        result = result - exp(-(X[i]^2 + X[i+1]^2 + 0.5*X[i]*X[i+1])/8)*cos(4*sqrt(X[i]^2 + X[i+1]^2 + 0.5*X[i]*X[i+1]))
    end    
    return result
end

Fobj3 (generic function with 1 method)

In [15]:
#Poniendo los datos
#Gerneraciones e individuos
batch_size = 1000
#Dimensionalidad
D = 10
#Límites de los parametros
bL = -5 .*ones(D)
bU = 5 .*ones(D)
value = 1 - D
error = 0.00001
#Argumentos de mutacion y cruza
F = 1.5
Cr = 0.3

0.3

In [16]:
initial = X0(batch_size, bL, bU)
i = 1
while true
    mutation = Vmut(initial,F,bL,bU,"d")
    crossed = Cross(initial,mutation,Cr)
    chosen = Selection(Fobj3,initial,crossed,"min")
    minimum, batch = Best(Fobj3, initial, "min")
    IJulia.clear_output(true)
    println("Iter: $i, value: $minimum, error: $(abs(minimum - value))")
    i = i+1
    if abs(minimum - value) <= error
        return minimum, batch
        break
    end
end

Iter: 1110, value: -8.999991812885337, error: 8.187114662661088e-6


(-8.999991812885337, [0.000222353, 3.04546e-5, -0.000281455, 0.000346211, -0.000123961, 0.000446228, 0.000211017, 0.000130815, 3.9249e-5, 0.000185738])

Valor de $-8.999991812885337$ con error de $8.187114662661088 \times 10^{-6}$

# Michalewicz function

In [27]:
function Fobj4(X)
    A = 1:length(X)
    return -sum(@.(sin(X)*sin(A .*(X.^2) / pi)^20))
end

Fobj4 (generic function with 1 method)

In [28]:
batch_size = 1000

D = 10

bL = 0 .*ones(D)
bU = pi .*ones(D)

value = -9.660151715641
error = 0.00001

F = 1.5
Cr = 0.3

0.3

In [29]:
initial = X0(batch_size, bL, bU)
i = 1
while true
    mutation = Vmut(initial,F,bL,bU,"d")
    crossed = Cross(initial,mutation,Cr)
    chosen = Selection(Fobj4,initial,crossed,"min")
    minimum, batch = Best(Fobj4, initial, "min")
    IJulia.clear_output(true)
    println("Iter: $i, value: $minimum, error: $(abs(minimum - value))")
    i = i+1
    if abs(minimum - value) <= error
        return minimum, batch
        break
    end
end

Iter: 371, value: -9.660142805687322, error: 8.909953677971316e-6


(-9.660142805687322, [2.20288, 1.57069, 1.28503, 1.92312, 1.72053, 1.5708, 1.45432, 1.75609, 1.65575, 1.57076])

Valor de $-9.660142805687322$ con error de $8.909953677971316 \times 10^{-6}$

# Rastrigin function

In [30]:
function Fobj5(X)
    return 10*D + dot(X,X) - 10*sum(cos.(2*pi.*X))
end

Fobj5 (generic function with 1 method)

In [31]:
batch_size = 1000

D = 30

bL = -5.12 .*ones(D)
bU = 5.12 .*ones(D)

value = 0
error = 0.00001

F = 1.5
Cr = 0.3

0.3

In [32]:
initial = X0(batch_size, bL, bU)
i = 1
while true
    mutation = Vmut(initial,F,bL,bU,"d")
    crossed = Cross(initial,mutation,Cr)
    chosen = Selection(Fobj5,initial,crossed,"min")
    minimum, batch = Best(Fobj5, initial, "min")
    IJulia.clear_output(true)
    println("Iter: $i, value: $minimum, error: $(abs(minimum - value))")
    i = i+1
    if abs(minimum - value) <= error
        return minimum, batch
        break
    end
end

Iter: 1736, value: 9.432414287857682e-6, error: 9.432414287857682e-6


(9.432414287857682e-6, [8.31847e-5, -5.07014e-5, 1.35595e-5, -6.56824e-5, -7.59348e-5, -5.54265e-5, -2.2288e-5, 6.57205e-5, 4.09393e-6, 4.84393e-5  …  3.93551e-5, 3.0389e-5, -3.47522e-5, 3.40505e-5, 1.78892e-5, 5.05835e-6, 5.17799e-6, 4.91575e-5, -2.45456e-5, -2.81089e-5])

Valor de $9.432414287857682 \times 10^{-6}$ con error de $9.432414287857682 \times 10^{-6}$

# Rosenbrock function

In [33]:
function Fobj6(X)
    suma = 0
    for i in i:length(X)-1
        suma = suma + (X[i]-1)^2 + 100*(X[i+1]-X[i]^2)^2
    end
    return suma
end

Fobj6 (generic function with 1 method)

In [34]:
batch_size = 1000

D = 30

bL = -30 .*ones(D)
bU = 30 .*ones(D)

value = 0
error = 0.01

F = 1.5
Cr = 0.3

0.3

In [36]:
initial = X0(batch_size, bL, bU)
i = 1
while true
    mutation = Vmut(initial,F,bL,bU,"d")
    crossed = Cross(initial,mutation,Cr)
    chosen = Selection(Fobj6,initial,crossed,"min")
    minimum, batch = Best(Fobj6, initial, "min")
    IJulia.clear_output(true)
    println("Iter: $i, value: $minimum, error: $(abs(minimum - value))")
    i = i+1
    if abs(minimum - value) <= error
        return minimum, batch
        break
    end
end

Iter: 30, value: 0.0, error: 0.0


(0.0, [-10.9094, -17.6329, 7.05599, 15.3731, 3.86197, -7.78909, 14.114, -19.4517, 3.51635, 22.192  …  -7.16919, -6.13115, 7.2436, 12.0927, 7.39719, -2.1563, -3.62979, 2.69906, -3.5985, 7.0688])

Valor de $0$ con error de $0$

# Shubert Function

In [11]:
function Fobj7(X)
    A = 1:5
    B = 2:6
    C = (sum(@.A*cos(B*X[1]+A)))*(sum(@.A*cos(B*X[2]+A)))
    return C
end

Fobj7 (generic function with 1 method)

In [10]:
batch_size = 1000

D = 2

bL = -10 .*ones(D)
bU = 10 .*ones(D)

value = -186.7309088310
error = 0.00001

F = 1.5
Cr = 0.3

0.3

In [12]:
initial = X0(batch_size, bL, bU)
i = 1
while true
    mutation = Vmut(initial,F,bL,bU,"d")
    crossed = Cross(initial,mutation,Cr)
    chosen = Selection(Fobj7,initial,crossed,"min")
    minimum, batch = Best(Fobj7, initial, "min")
    IJulia.clear_output(true)
    println("Iter: $i, value: $minimum, error: $(abs(minimum - value))")
    i = i+1
    if abs(minimum - value) <= error
        return minimum, batch
        break
    end
end

Iter: 263, value: -186.73090595794514, error: 2.87305485358047e-6


(-186.73090595794514, [-1.4251, -7.08349])

Valor de $-186.73090595794514$ con error de $2.87305485358047 \times 10^{-6}$

# Sinusoidal problem

In [13]:
function Fobj8(X)
    A=2.5
    B=5
    z=30
    r1=1
    r2=1
    for i in 1:length(X)
        r1 = r1*sin(X[i]-z)
        r2 = r2*sin(B*(X[i]-z))
    end
    return -A * r1 - r2
end

Fobj8 (generic function with 1 method)

In [14]:
batch_size = 1000

D = 2

bL = 0 .*ones(D)
bU = 180 .*ones(D)

value = -(2.5 + 1)
error = 0.01

F = 1.5
Cr = 0.3

0.3

In [15]:
initial = X0(batch_size, bL, bU)
i = 1
while true
    mutation = Vmut(initial,F,bL,bU,"d")
    crossed = Cross(initial,mutation,Cr)
    chosen = Selection(Fobj8,initial,crossed,"min")
    minimum, batch = Best(Fobj8, initial, "min")
    IJulia.clear_output(true)
    println("Iter: $i, value: $minimum, error: $(abs(minimum - value))")
    i = i+1
    if abs(minimum - value) <= error
        return minimum, batch
        break
    end
end

Iter: 10, value: -3.499374523647619, error: 0.0006254763523809714


(-3.499374523647619, [169.796, 144.673])

Valor de $-3.499374523647619$ con error de $0.0006254763523809714$

# Sphere function

In [16]:
function Fobj9(X)
    return X'*X
end

Fobj9 (generic function with 1 method)

In [17]:
batch_size = 1000

D = 2

bL = -5.12 .*ones(D)
bU = 5.12 .*ones(D)

value = 0
error = 0.00001

F = 1.5
Cr = 0.3

0.3

In [18]:
initial = X0(batch_size, bL, bU)
i = 1
while true
    mutation = Vmut(initial,F,bL,bU,"d")
    crossed = Cross(initial,mutation,Cr)
    chosen = Selection(Fobj9,initial,crossed,"min")
    minimum, batch = Best(Fobj9, initial, "min")
    IJulia.clear_output(true)
    println("Iter: $i, value: $minimum, error: $(abs(minimum - value))")
    i = i+1
    if abs(minimum - value) <= error
        return minimum, batch
        break
    end
end

Iter: 20, value: 1.181687084986292e-6, error: 1.181687084986292e-6


(1.181687084986292e-6, [-0.000883779, -0.000632948])

Valor de $1.181687084986292 \times 10^{-6}$ con error de $1.181687084986292 \times 10^{-6}$

# Zakharov function

In [19]:
function Fobj10(X)
    n = 1:length(X)
    A = sum(0.5.*n.*X)
    return X'*X + A^2 + A^4
end

Fobj10 (generic function with 1 method)

In [20]:
batch_size = 1000

D = 2

bL = -5 .*ones(D)
bU = 10 .*ones(D)

value = 0
error = 0.00001

F = 1.5
Cr = 0.3

0.3

In [21]:
initial = X0(batch_size, bL, bU)
i = 1
while true
    mutation = Vmut(initial,F,bL,bU,"d")
    crossed = Cross(initial,mutation,Cr)
    chosen = Selection(Fobj10,initial,crossed,"min")
    minimum, batch = Best(Fobj10, initial, "min")
    IJulia.clear_output(true)
    println("Iter: $i, value: $minimum, error: $(abs(minimum - value))")
    i = i+1
    if abs(minimum - value) <= error
        return minimum, batch
        break
    end
end

Iter: 20, value: 7.127470279747377e-6, error: 7.127470279747377e-6


(7.127470279747377e-6, [0.00126572, 0.00131531])

Valor de $7.127470279747377 \times 10^{-6}$ con error de $7.127470279747377 \times 10^{-6}$